In [10]:
import warnings
warnings.filterwarnings('ignore')
import os
import awkward as ak
import uproot
import hist
import numpy as np
from coffea import processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea.processor import dict_accumulator, list_accumulator
from coffea import nanoevents

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
plt.style.use(hep.style.ROOT)

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'medium',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

#line thickness
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 5
import itertools
import json

# for making fancy 2d histograms
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from trig_eff_process import create_baseline_selection_mask

In [4]:
events = nanoevents.NanoEventsFactory.from_root(
    "root://cmseos.fnal.gov//store/user/lpcdihiggsboost/NanoAOD_v12_ParT/rkansal/2022/HHbbtt/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_LHEweights_TuneCP5_13p6TeV_powheg-pythia8/GluGlutoHHto2B2Tau_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV/241028_235514/0000/Run3Summer22NanoAODv12_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

In [33]:
from coffea import processor
import numpy as np
import awkward as ak

class TriggerSoupProcessor(processor.ProcessorABC):
    """
    Calculate the trigger efficiency of the soup
    """
    def __init__(self, trig_soup, trig_vars, baseline_selection_mask):
        self.trig_soup = trig_soup
        self.trig_vars = trig_vars
        self.baseline_selection_mask = baseline_selection_mask
        self._accumulator = processor.dict_accumulator({
            'num_pass': 0,
            'num_total': 0,
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        # Compute the logical OR of all triggers in the soup
        # Initialize trigger_or_mask with False for all events
        trigger_or_mask = ak.zeros_like(events.event, dtype=bool)

        # Accumulate the logical OR without in-place modification
        for trig in self.trig_soup:
            trigger_or_mask = trigger_or_mask | events.HLT[trig]

        # Apply the baseline selection mask
        if callable(self.baseline_selection_mask):
            baseline_mask = self.baseline_selection_mask(events)
        else:
            baseline_mask = events[self.baseline_selection_mask]

        # Calculate the number of events passing both baseline and trigger
        num_pass = ak.sum(baseline_mask & trigger_or_mask)
        num_total = ak.sum(baseline_mask)

        # Update the accumulator
        self._accumulator['num_pass'] += num_pass
        self._accumulator['num_total'] += num_total

        return self.accumulator

    def postprocess(self, accumulator):
        # Calculate and add efficiency to the accumulator
        num_pass = accumulator['num_pass']
        num_total = accumulator['num_total']
        efficiency = num_pass / num_total if num_total > 0 else 0
        accumulator['efficiency'] = efficiency
        return accumulator


In [34]:
def baseline_selection_mask(events):
    jets = events['Jet']  # Assuming jet information is stored under the key 'Jet'

    # Ensure that each event has at least four jets
    has_four_jets = ak.num(jets.pt) >= 4

    # Initialize a mask with False for events with fewer than four jets
    leading_jet_pt_mask = has_four_jets

    # For events with at least four jets, apply pT thresholds to the leading jets
    if ak.any(has_four_jets):
        # Create a mask for the pT thresholds
        pt_mask = (
            (jets.pt[has_four_jets][:, 0] > 105) &
            (jets.pt[has_four_jets][:, 1] > 88) &
            (jets.pt[has_four_jets][:, 2] > 76) &
            (jets.pt[has_four_jets][:, 3] > 15)
        )
        # Update the leading_jet_pt_mask with the pT conditions
        leading_jet_pt_mask = ak.where(has_four_jets, pt_mask, False)
    else:
        leading_jet_pt_mask = ak.zeros_like(has_four_jets, dtype=bool)

    return leading_jet_pt_mask


In [35]:
trig_soup = [
        'AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35',
        'AK8PFJet425_SoftDropMass40',
        'QuadPFJet105_88_76_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
        'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65',
        'QuadPFJet103_88_75_15_PFBTagDeepJet_1p3_VBF2',
        'QuadPFJet103_88_75_15_DoublePFBTagDeepJet_1p3_7p7_VBF1',
        'QuadPFJet70_50_40_35_PFBTagParticleNet_2BTagSum0p65'
    ]
trig_vars = {
    'ht': {
        'label': "H_{T} [GeV]",
        'axis': hist.axis.Regular(bins=100, start=0, stop=2000, name="ht", label="H_{T} [GeV]"),
        'proc': lambda events: ak.sum(events.FatJet.pt, axis=1)
    },
    'pt': {
        'label': "Leading Jet $p_{T}$ [GeV]",
        'axis': hist.axis.Regular(bins=30, start=0, stop=1200, name="pt", label="Leading Jet $p_{T}$ [GeV]"),
        'proc': lambda events: ak.fill_none(ak.pad_none(events.FatJet.pt, 1, clip=True)[:, 0], np.nan)
    },
    'msd': {
        'label': "Leading Jet $m_{SD}$ [GeV]",
        'axis': hist.axis.Regular(bins=10, start=40, stop=200, name="msd", label="Leading Jet $m_{SD}$ [GeV]"),
        'proc': lambda events: ak.fill_none(ak.pad_none(events.FatJet.msoftdrop, 1, clip=True)[:, 0], np.nan)
    },
    'num_ak4': {
        'label': "Number of AK4 Jets",
        'axis': hist.axis.Integer(0, 20, name="num_ak4", label="Number of AK4 Jets"),
        'proc': lambda events: ak.num(events.Jet)
    },
    'gen_H_pt': {
        'label': "Gen Higgs pT [GeV]",
        'axis': hist.axis.Regular(bins=30, start=0, stop=1200, name="gen_H_pt", label="Gen Higgs pT [GeV]"),
        'proc': lambda events: events.HTXS.Higgs_pt
    },
    'particleNet_XbbVsQCD':{
        'label': "Leading Particle Net TXbb score",
        'axis': hist.axis.Regular(bins=30, start=0, stop=1, name="particleNet_XbbVsQCD", label="Leading Particle Net TXbb score"),
        'proc': lambda events: ak.fill_none(
            ak.pad_none(events.FatJet.particleNet_XbbVsQCD, 1, clip=True)[:, 0],
            np.nan
        )
    }
}
processor_instance = TriggerSoupProcessor(trig_soup, trig_vars, baseline_selection_mask)

In [36]:
out = processor_instance.process(events)

ValueError: cannot broadcast RegularArray of size 25281 with RegularArray of size 25300

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.5/src/awkward/_util.py#L920)

In [ ]:
events.HLT['AK8PFJet250_SoftDropMass40_PFAK8ParticleNetBB0p35']